<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Pui/American_Put_jax_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# American Put

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
numthreads=10
numpaths=60000
dim=35
rng = jax.random.PRNGKey(2)
rng, key = jax.random.split(rng)
weights=random.normal(key,(dim,))

X=random.normal(key,(numpaths,dim))
Y=jnp.matmul(X,weights)+random.normal(key,(numpaths,))*.6
# print(weights)
print(jax.numpy.linalg.lstsq(X,Y,rcond=-1)[0])

[ 2.2714288   0.04714125 -0.3861555   1.2990805   0.05353612 -1.148216
  1.3288654   0.9705746   0.86686456 -0.714077    0.15883517 -0.03464958
  1.7595551   0.85336447 -0.5681019   0.14077947  0.73238677 -0.34808257
 -0.46217936 -0.6631018  -1.3822023   0.28682745 -0.6331234  -0.3799799
  0.70073104  1.5497264   0.28652    -0.7819917  -1.122198    0.28971723
 -0.45474672 -0.9023748   1.9462991   0.09517366  0.07593049]


In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
numthreads=10
numpaths=60000
dim=35
rng = jax.random.PRNGKey(2)
rng, key = jax.random.split(rng)
weights=random.normal(key,(dim,))

X=random.normal(key,(numpaths,dim))
Y=jnp.matmul(X,weights)+random.normal(key,(numpaths,))*.6
# print(weights)
jnp.linalg.inv(X.T @ X)@(X.T@Y)

DeviceArray([ 2.271427  ,  0.04714175, -0.3861555 ,  1.2990808 ,
              0.05353574, -1.1482165 ,  1.3288662 ,  0.9705749 ,
              0.8668647 , -0.7140763 ,  0.15883441, -0.03464897,
              1.7595538 ,  0.85336465, -0.5681017 ,  0.1407793 ,
              0.7323865 , -0.3480823 , -0.4621794 , -0.6631015 ,
             -1.3822019 ,  0.28682724, -0.6331228 , -0.3799796 ,
              0.7007311 ,  1.5497264 ,  0.28652024, -0.7819918 ,
             -1.1221977 ,  0.28971738, -0.45474735, -0.9023752 ,
              1.9462987 ,  0.09517355,  0.07593112], dtype=float32)

In [ ]:
import numpy as np
X = np.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2],
                  [1.08**3,1.07**3,0.97**3,0.77**3,0.84**3]]).transpose()
Y = np.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(jax.numpy.linalg.lstsq(X,Y,rcond=-1)[0])
print(X.dot(np.linalg.lstsq(X,Y,rcond=-1)[0]))

[  49.12175  -162.2587    178.01677   -64.704094]
[0.01256467 0.05102041 0.17311223 0.18914178 0.08271131]


In [ ]:
import numpy as np
X = np.array([[1,1,1,1,1],
                   [1.08,1.07,0.97,0.77,0.84],
                   [1.08**2,1.07**2,0.97**2,0.77**2,0.84**2],
                  [1.08**3,1.07**3,0.97**3,0.77**3,0.84**3]]).transpose()
Y = np.array([0, 0.07*0.94176, 0.18*0.94176, 0.2*0.94176, 0.09*0.94176])

print(jnp.linalg.inv(X.T @ X)@(X.T@Y))
print(X.dot(jnp.linalg.inv(X.T @ X)@(X.T@Y)))

[ 13.498047 -45.09375   50.671875 -18.945312]
[0.03483438 0.0531416  0.14340137 0.17005371 0.14440937]


In [2]:
# change from CF matrix to CF array
# put
# large numpaths & numsteps to test price here
# use lax.scan
# replace X=0 and Y=0 for OTM
# add inner loop for multiple batches: can have more numpaths by increasing numbatches
# if run out of memory, try restarting the session

import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        dx = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
  numbatches = 10 # if numbatches is larger, max numpaths can be larger. Note: the larger the numbatches, the longer the computation will take
  batched_b = jax.vmap(Brownian_motion, in_axes=(0, None, None, None, None, None))
  
  # out = batch_simple(keys, initial_stocks, numsteps, drift, cov, T)
  out_avg = jnp.mean(batched_b(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

  dt = T / numsteps
  ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
  CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period

  def for_loop(numsteps, r, K, dt, ITM_matrix, CF_array, numpaths, numbatches):
    def body_fun(val, i):
      ITM_boolean = ITM_matrix[:,-i]
      X = jnp.where(ITM_boolean, out_avg[:,-i], 0) # if ITM, use out_avg, if not, use 0
      val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
      Y = jnp.where(ITM_boolean, val, 0) # if ITM, Y = CF array, if not, use 0
      X_matrix = jnp.array([X**0,X**1,X**2,X**3]).transpose()

      ECV = jnp.zeros(numpaths) # initialize ECV array with 0's
      X_matrix_3D = X_matrix.reshape((numbatches, -1, 4))
      Y_3D = Y.reshape((numbatches, -1))
      ITM_boolean_3D = ITM_boolean.reshape((numbatches, -1))

      # inner loop (more batches so that numpaths can be larger without running out of memory)
      def ECV_fun(val_2, i_2):
        ECV_sub = X_matrix_3D[i_2].dot(jax.numpy.linalg.lstsq(X_matrix_3D[i_2], Y_3D[i_2], rcond=-1)[0]) # predicted Y - ECV - expected continue value
        ECV_sub = jnp.where(ITM_boolean_3D[i_2], ECV_sub, 10000) # replace not ITM ECV by 10000 (so that K-X < ECV)
        val_2 = jax.lax.dynamic_update_slice(val_2,
                                             ECV_sub,
                                             ((numpaths*i_2/numbatches).astype(int),)) # update the part for this batch in ECV array
        return val_2, i_2

      counts_2 = jnp.arange(numbatches)
      ECV, _ =  jax.lax.scan(ECV_fun, ECV, counts_2)

      # update CF_array
      KX_larger_than_ECV = (K-X > ECV)
      val = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*val
      return val, i

    counts = jnp.arange(2, numsteps+1)
    CF_array, _ =  jax.lax.scan(body_fun, CF_array, counts)
    return (CF_array * jnp.exp(-r * dt)).mean()

  CF_out = for_loop(numsteps, r[0], K, dt, ITM_matrix, CF_array, numpaths, numbatches)
  return CF_out

numstocks = 1
numpaths = 100000
numsteps = 50

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
drift = jnp.array([0.05]*numstocks)
r = drift

cov = jnp.identity(numstocks)*0.25*0.25
initial_stocks = jnp.array([1.]*numstocks) # must be float

T = 1.0
K = 1.0

#fast_simple = jax.jit(Brownian_motion, static_argnums=2)

keys = jax.random.split(key, numpaths)
#batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

fast_ov = jax.jit(optionvalueavg, static_argnums=(2, 8))

# option price
print(fast_ov(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(fast_ov(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# delta
goptionvalueavg = jax.grad(fast_ov,argnums=1)
print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
%timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

0.080257475
10 loops, best of 5: 178 ms per loop
[-0.40965283]
1 loop, best of 5: 2.24 s per loop


In [ ]:
# # change from CF matrix to CF array
# # put
# # large numpaths & numsteps to test price here
# # use lax.scan
# # replace X=0 and Y=0 for OTM
# # add inner loop for multiple batches: can have more numpaths by increasing numbatches
# # if run out of memory, try restarting the session

# import jax
# import jax.numpy as jnp
# from jax import random
# from jax import jit
# import numpy as np

# def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
#     stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
#     stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
#                             jax.ops.index[0],         # initialization of stock prices
#                             initial_stocks)
#     noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
#     sigma = jnp.diag(cov) ** 0.5
#     dt = T / numsteps
#     def time_step(t, val):
#         dx = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
#         val = jax.ops.index_update(val,
#                             jax.ops.index[t],
#                             val[t-1] * dx)
#         return val
#     return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

# def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths):  
#   batched_b = jax.vmap(Brownian_motion, in_axes=(0, None, None, None, None, None))
#   out_avg = jnp.mean(batched_b(keys, initial_stocks, numsteps, drift, cov, T), axis=2)

#   dt = T / numsteps
#   ITM_matrix = out_avg <= K # matrix that is true if ITM, else false
#   CF_array = jnp.maximum(K-out_avg[:,-1], 0) # array that will store discounted cash flows for the next time period

#   def for_loop(numsteps, r, K, dt, ITM_matrix, CF_array):
#     def body_fun(val, i):
#       ITM_boolean = ITM_matrix[:,-i]
#       X = jnp.where(ITM_boolean, out_avg[:,-i], 0) # if ITM, use out_avg, if not, use 0
#       val = (val * jnp.exp(-r * dt)) # discount CF_array by one step
#       Y = jnp.where(ITM_boolean, val, 0) # if ITM, Y = CF array, if not, use 0
#       X_matrix = jnp.array([X**0,X**1,X**2,X**3]).transpose()
      
#       #ECV = X_matrix.dot(jax.numpy.linalg.lstsq(X_matrix, Y, rcond=-1)[0]) # predicted Y - ECV - expected continue value
#       ECV = X_matrix.dot(jnp.linalg.inv(X_matrix.T @ X_matrix)@(X_matrix.T@Y)) # predicted Y - ECV - expected continue value
#       ECV = jnp.where(ITM_boolean, ECV, 10000) # replace not ITM ECV by 10000 (so that K-X < ECV)

#       # update CF_array
#       KX_larger_than_ECV = (K-X > ECV)
#       val = KX_larger_than_ECV*(K-X) + (1-KX_larger_than_ECV)*val
#       return val, i

#     counts = jnp.arange(2, numsteps+1)
#     CF_array, _ =  jax.lax.scan(body_fun, CF_array, counts)
#     return (CF_array * jnp.exp(-r * dt)).mean()

#   CF_out = for_loop(numsteps, r[0], K, dt, ITM_matrix, CF_array)
#   return CF_out
  
# numstocks = 1
# numpaths = 100000
# numsteps = 200

# rng = jax.random.PRNGKey(np.random.randint(10000))
# rng, key = jax.random.split(rng)
# drift = jnp.array([0.05]*numstocks)
# r = drift

# cov = jnp.identity(numstocks)*0.25*0.25
# initial_stocks = jnp.array([1.]*numstocks) # must be float

# T = 1.0
# K = 1.0

# #fast_simple = jax.jit(Brownian_motion, static_argnums=2)

# keys = jax.random.split(key, numpaths)
# #batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))

# fast_ov = jax.jit(optionvalueavg, static_argnums=(2, 8))

# # option price
# print(fast_ov(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(fast_ov(key, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

# # delta
# goptionvalueavg = jax.grad(fast_ov,argnums=1)
# print(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))
# %timeit(goptionvalueavg(keys, initial_stocks, numsteps, drift, r, cov, K, T, numpaths))

0.041642092
10 loops, best of 5: 143 ms per loop
[-0.73047256]
10 loops, best of 5: 359 ms per loop
